In [3]:
import CS_Indicators as CS
from brix import Indicator, Handler
import PreCompOsmNet
import Simulation
import geopandas as gpd
import pandas as pd

In [4]:
table_name='providence_test'

### Load the Saved Data

In [5]:
geogrid=gpd.read_file('tables/{}/geogrid.geojson'.format(table_name))
zones=gpd.read_file('tables/{}/zones.geojson'.format(table_name))
zones['GEOID']=zones['GEOID'].astype(int)
zones=zones.set_index('GEOID')
simpop_df=pd.read_csv('tables/{}/simpop_df.csv'.format(table_name))

### LODES industry is not useful- look up NAICS from wac instead
- TODO: do this in the script which prepares the USA data

In [28]:
import random
naics_cols=[col for col in zones.columns if 'naics' in col]
simpop_df['naics']=0
for ind, row in simpop_df.iterrows():
    if zones.loc[row['work_geoid'], 'emp_total']>0:
        simpop_df.loc[ind, 'naics']=random.choices(naics_cols, weights=zones.loc[row['work_geoid'], naics_cols].values, k=1)[0]
    else:
        simpop_df.loc[ind, 'naics']=random.choices(naics_cols, k=1)[0]

In [31]:
simpop_df=simpop_df.drop('industry', axis=1)

### Create the mobility system

In [32]:
networks, mode_dicts, route_lengths=PreCompOsmNet.create_2_scale_osmnx_network(
    zones.loc[zones['sim_area']], zones.loc[zones['model_area']],
    add_modes=[{'name': 'walk', 'speed': 4800/3600},
               {'name': 'cycle', 'speed': 14000/3600},
               {'name': 'pt', 'speed': 25000/3600}])
modes={mode: Simulation.Mode(mode_dicts[mode]) for mode in mode_dicts}
mob_sys=Simulation.MobilitySystem(modes=modes, networks=networks)

getting internal roads
Buffer : 200
Doesnt contain external link types
Buffer : 400
getting external roads
	 Simplified from 2143 to 1891 edges and 929 to 750 nodes
	 Pre-computing paths


In [33]:
import numpy as np

class Default_Mode_Choice_model():
    def __init__(self):
        pass
    
    def predict_modes(self, all_trips_df):
        options=['drive', 'cycle', 'walk', 'pt']
        all_trips_df['mode']='drive'
        ind_u1500=all_trips_df['route_distance']<1500
        ind_1500_5k=((all_trips_df['route_distance']>1500)&(all_trips_df['route_distance']<5000))
        ind_5k_10k=((all_trips_df['route_distance']>5000)&(all_trips_df['route_distance']<10000))
        ind_10kplus=all_trips_df['route_distance']>10000

        all_trips_df.loc[ind_u1500==True, 'mode']=np.random.choice(
            options, size=sum(ind_u1500), 
            replace=True, p=[0.1, 0.2, 0.6, 0.1])

        all_trips_df.loc[ind_1500_5k==True, 'mode']=np.random.choice(
            options, size=sum(ind_1500_5k), 
            replace=True, p=[0.25, 0.15, 0.3, 0.3])

        all_trips_df.loc[ind_5k_10k==True, 'mode']=np.random.choice(
            options, size=sum(ind_5k_10k), 
            replace=True, p=[0.6, 0.04, 0.01, 0.35])

        all_trips_df.loc[ind_10kplus==True, 'mode']=np.random.choice(
            options, size=sum(ind_10kplus), 
            replace=True, p=[0.9, 0.01, 0.01, 0.08])
        return all_trips_df

mode_descriptions = [{"name": 'drive',
                    'color': "#e41a1d"},
                     {"name": 'cycle',
                    'color': "#377eb8"},
                     {"name": 'walk',
                    'color': "#4daf4a"},
                     {"name": 'pt',
                    'color': "#ffff33"},
                    ]

### Create and run the Cityscope model

In [34]:
H=Handler(table_name=table_name)
H.reset_geogrid_data()

https://cityio.media.mit.edu/api/tables/list/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/GEOGRID/ Attempt: 0
Proceeding without reviewing GEOGRIDDATA
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
GEOGRIDDATA successfully updated: 6i37RGg4CAEQAvtSa6fnnPaPCRjYMxc65r5q6Spcd5XY


In [35]:
# d=CS.Density_Indicator(zones=zones)
# p=CS.Proximity_Indicator(zones=zones, geogrid=geogrid)


m=CS.Mobility_indicator(zones, geogrid, table_name, simpop_df, mob_sys,
#                        mode_choice_model=new_mode_choice_model,
#                        mode_descriptions=mode_descriptions,
                       route_lengths=route_lengths)
H.add_indicators([
#     d, p, 
    m
])

# H.add_indicator(m)

Init simulation
Getting internal nodes


/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/CS_Indicators.py:606: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids=grid_zones['geometry'].centroid


Finding closest nodes to every zone centroid


/Users/doorleyr/anaconda3/envs/brix/lib/python3.9/site-packages/osmnx/distance.py:426: UserWarning: The `get_nearest_nodes` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


Computing zone-zone dist mat
https://cityio.media.mit.edu/api/table/providence_test/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:406: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.275039
	 trips post took 0:00:01.711424


In [36]:
# self=m
# import datetime
# print('Starting MM Update')
# start_calc=datetime.datetime.now()
# new_simpop=self.geogrid_updates(geogrid_data)
# new_simpop_df=pd.DataFrame(new_simpop)
# combined_simpop=self.base_simpop_df.append(new_simpop_df)
# sample_simpop_df=combined_simpop.sample(min(self.N_max, len(combined_simpop)))
# od_output, deckgl_trips, ind=self.simulate(sample_simpop_df)

In [37]:
m.sim.person_attributes

['age', 'earnings', 'naics', 'mode']

In [39]:
geogrid_data=H.get_geogrid_data()
m.return_indicator(geogrid_data)

https://cityio.media.mit.edu/api/table/providence_test/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table


/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:406: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.364494
	 trips post took 0:00:01.723369


{'name': 'Active Mobility',
 'raw_value': 0.28,
 'value': 0.28,
 'ref_value': 0.1,
 'viz_type': 'radar'}

In [40]:
H.listen()

Table URL: https://cityscope.media.mit.edu/CS_cityscopeJS/?cityscope=providence_test
Testing indicators
https://cityio.media.mit.edu/api/table/providence_test/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:406: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.291986
	 trips post took 0:00:01.703889
Performing initial update
Updating geogrid_data
https://cityio.media.mit.edu/api/table/providence_test/GEOGRIDDATA/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
Updating table with hash: 6i37RGg4CAEQAvtSa6fnnPaPCRjYMxc65r5q6Spcd5XY
https://cityio.media.mit.edu/api/table/providence_test/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table


/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:406: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.466531
	 trips post took 0:00:01.713894
Done with update
Local grid hash: 6i37RGg4CAEQAvtSa6fnnPaPCRjYMxc65r5q6Spcd5XY
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
Waking up!
Updating geogrid_data
https://cityio.media.mit.edu/api/table/providence_test/GEOGRIDDATA/ Attempt: 0
Updating table with hash: MHba4

/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:406: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.942851
	 trips post took 0:00:01.565119
Done with update
Local grid hash: MHba4PUCt1Nkc58EBcU7HnUoW9pRbpJjp2BinBXUnca
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Atte

/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:406: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.802595
	 trips post took 0:00:01.605264
Done with update
Local grid hash: EQC3X85MgysCG59nbMN443eeSdTXHnyann2Tt3EdgZK9
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Att

KeyboardInterrupt: 